In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install optuna xgboost lightgbm "mlflow<3"

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/29.0 MB ? eta -:--:--
   ---- ----------------------------------- 3.4/29.0 MB 23.1 MB/s eta 0:00:02
   --------------- ------------------------ 11.0/29.0 MB 30.6 MB/s eta 0:00:01
   --------------------------- ------------ 20.2/29.0 MB 35.0 MB/s eta 0:00:01
   ------------------------------------- -- 27.5/29.0 MB 34.8 MB/s eta 0:00:01
   ---------------------------------------- 29.0/29.0 MB 32.5 MB/s  0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----------------------------------- ---- 5.5/6.3 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 24.2 MB/s  0:00:00
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   -------- ------------------------------- 5.5/25.2 MB 26.6 MB/s eta 0:00:01
   -------------- ------------------------- 8.9/25.2 MB 22.6 MB/s eta 0:00:01
   ------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ASSUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
# %cd "{base_folder}"

In [4]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))


In [7]:
import sqlite3
import pandas as pd
from pathlib import Path

BASE_DIR = Path("..")          # project root
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
conn = sqlite3.connect(BASE_DIR / "notebooks" / "housing.db")

housing = pd.read_sql_query(
    """
    SELECT
        b.block_id,
        b.longitude,
        b.latitude,
        s.housing_median_age,
        s.total_rooms,
        s.total_bedrooms,
        s.population,
        s.households,
        s.median_income,
        s.median_house_value,
        op.name AS ocean_proximity
    FROM block AS b
    JOIN block_housing_stats AS s
        ON s.block_id = b.block_id
    JOIN ocean_proximity AS op
        ON op.ocean_proximity_id = b.ocean_proximity_id
    ORDER BY b.block_id
    """,
    conn,
)
conn.close()

housing.head()

,block_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [ ]:
# =============================================================================
# FULL PIPELINE with OPTUNA
# - Build preprocessing
# - Stratified train/test split
# - Train & log 4 models WITHOUT PCA (Ridge, HGB, XGBoost, LightGBM)
# - Train & log 4 models WITH PCA (preprocessing + PCA(0.95) + model)
# - Pick GLOBAL best among 8 models by Test MAE
# - Save, load, and compare the global best model
# =============================================================================

import time
import os
import numpy as np
import pandas as pd

from dotenv import load_dotenv

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor

import mlflow
from mlflow.models import infer_signature
import joblib

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import optuna
from optuna.samplers import TPESampler

from sklearn.base import clone

# Shared components
from housing_pipeline import (
    build_preprocessing,
    make_estimator_for_name,
)

start_time = time.monotonic()
optuna.logging.set_verbosity(optuna.logging.WARNING)


# =============================================================================
# STEP 1: Build Full ML Preprocessing Pipeline
# =============================================================================

preprocessing = build_preprocessing()
print("✓ STEP 1: Preprocessing pipeline created.")


# =============================================================================
# STEP 2: Split Data into Stratified Train and Test Sets
# =============================================================================

housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0, 1.5, 3.0, 4.5, 6, np.inf],
    labels=[1, 2, 3, 4, 5],
)

train_set, test_set = train_test_split(
    housing,
    test_size=0.20,
    stratify=housing["income_cat"],
    random_state=42,
)

for df in (train_set, test_set):
    df.drop("income_cat", axis=1, inplace=True)

X_train = train_set.drop(["block_id", "median_house_value"], axis=1).copy()
y_train = train_set["median_house_value"].copy()

X_test = test_set.drop(["block_id", "median_house_value"], axis=1).copy()
y_test = test_set["median_house_value"].copy()

print(f"✓ STEP 2: Stratified split done. Train size: {len(X_train)}, Test size: {len(X_test)}")


# =============================================================================
# STEP 3: Configure MLflow
# =============================================================================

load_dotenv(
    dotenv_path=Path("..") / "notebooks" / ".env",
    override=True
)


MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

if MLFLOW_TRACKING_USERNAME:
    os.environ["MLFLOW_TRACKING_USERNAME"] = MLFLOW_TRACKING_USERNAME
if MLFLOW_TRACKING_PASSWORD:
    os.environ["MLFLOW_TRACKING_PASSWORD"] = MLFLOW_TRACKING_PASSWORD

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("median_house_pricing_multi_model_optuna")

print("✓ STEP 3: MLflow configured.")


# =============================================================================
# STEP 4: Define Optuna Objective Functions (NO PCA)
# =============================================================================

def objective_ridge(trial, preprocessing, X_train, y_train):
    alpha = trial.suggest_float("ridge__alpha", 0.1, 100.0, log=True)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(preprocessing_clone, Ridge(alpha=alpha))
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_hgb(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("hgb__learning_rate", 0.05, 0.2)
    max_depth = trial.suggest_int("hgb__max_depth", 3, 8)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        HistGradientBoostingRegressor(
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=42
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_xgb(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("xgb__learning_rate", 0.05, 0.2)
    max_depth = trial.suggest_int("xgb__max_depth", 3, 8)
    n_estimators = trial.suggest_int("xgb__n_estimators", 100, 300, step=50)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        XGBRegressor(
            objective="reg:squarederror",
            random_state=42,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            tree_method="hist",
            n_jobs=-1,
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_lgbm(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("lgbm__learning_rate", 0.05, 0.2)
    num_leaves = trial.suggest_int("lgbm__num_leaves", 20, 80)
    n_estimators = trial.suggest_int("lgbm__n_estimators", 100, 300, step=50)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        LGBMRegressor(
            random_state=42,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            n_jobs=-1,
            verbose=-1,
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


# =============================================================================
# STEP 5: Run Optuna Studies for Each Model (NO PCA)
# =============================================================================

model_names = ["ridge", "histgradientboosting", "xgboost", "lightgbm"]
objective_functions = {
    "ridge": objective_ridge,
    "histgradientboosting": objective_hgb,
    "xgboost": objective_xgb,
    "lightgbm": objective_lgbm,
}

results = {}

for name in model_names:
    print(f"\n{'='*80}")
    print(f"Optimizing {name.upper()} (NO PCA) - 10 trials")
    print(f"{'='*80}")

    study = optuna.create_study(
        direction="minimize",
        sampler=TPESampler(seed=42),
        study_name=f"{name}_study"
    )

    study.optimize(
        lambda trial: objective_functions[name](trial, preprocessing, X_train, y_train),
        n_trials=10,
        show_progress_bar=True
    )

    cv_mae = study.best_value
    print(f"\nBest {name.upper()} CV MAE: ${cv_mae:,.2f}")
    print(f"Best params: {study.best_params}")

    best_params = study.best_params
    preprocessing_clone = clone(preprocessing)

    if name == "ridge":
        final_model = make_pipeline(
            preprocessing_clone,
            Ridge(alpha=best_params["ridge__alpha"])
        )
    elif name == "histgradientboosting":
        final_model = make_pipeline(
            preprocessing_clone,
            HistGradientBoostingRegressor(
                learning_rate=best_params["hgb__learning_rate"],
                max_depth=best_params["hgb__max_depth"],
                random_state=42
            )
        )
    elif name == "xgboost":
        final_model = make_pipeline(
            preprocessing_clone,
            XGBRegressor(
                objective="reg:squarederror",
                random_state=42,
                n_estimators=best_params["xgb__n_estimators"],
                learning_rate=best_params["xgb__learning_rate"],
                max_depth=best_params["xgb__max_depth"],
                tree_method="hist",
                n_jobs=-1,
            )
        )
    elif name == "lightgbm":
        final_model = make_pipeline(
            preprocessing_clone,
            LGBMRegressor(
                random_state=42,
                n_estimators=best_params["lgbm__n_estimators"],
                learning_rate=best_params["lgbm__learning_rate"],
                num_leaves=best_params["lgbm__num_leaves"],
                n_jobs=-1,
                verbose=-1,
            )
        )

    final_model.fit(X_train, y_train)

    y_pred = final_model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    print(f"{name} (no PCA) Test MAE: ${test_mae:,.2f}")

    results[name] = {"pipeline": final_model, "test_mae": test_mae, "cv_mae": cv_mae}

    with mlflow.start_run(run_name=f"{name}_baseline_optuna"):
        mlflow.log_param("model_family", name)
        mlflow.log_param("uses_pca", False)
        mlflow.log_params(best_params)
        mlflow.log_metric("cv_MAE", cv_mae)
        mlflow.log_metric("test_MAE", test_mae)

        signature = infer_signature(X_train, final_model.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=final_model,
            artifact_path="housing_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{name}_pipeline_optuna",
        )

print("\n✓ STEP 5: All 4 baseline models optimized and logged.")


# =============================================================================
# STEP 6: PCA Optuna Objectives
# =============================================================================

def objective_ridge_pca(trial, preprocessing, X_train, y_train):
    alpha = trial.suggest_float("ridge__alpha", 0.1, 100.0, log=True)
    pca_components = trial.suggest_float("pca__n_components", 0.90, 0.99)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(preprocessing_clone, PCA(n_components=pca_components), Ridge(alpha=alpha))
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_hgb_pca(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("hgb__learning_rate", 0.05, 0.2)
    max_depth = trial.suggest_int("hgb__max_depth", 3, 8)
    pca_components = trial.suggest_float("pca__n_components", 0.90, 0.99)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        PCA(n_components=pca_components),
        HistGradientBoostingRegressor(
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=42
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_xgb_pca(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("xgb__learning_rate", 0.05, 0.2)
    max_depth = trial.suggest_int("xgb__max_depth", 3, 8)
    n_estimators = trial.suggest_int("xgb__n_estimators", 100, 300, step=50)
    pca_components = trial.suggest_float("pca__n_components", 0.90, 0.99)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        PCA(n_components=pca_components),
        XGBRegressor(
            objective="reg:squarederror",
            random_state=42,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            tree_method="hist",
            n_jobs=-1,
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


def objective_lgbm_pca(trial, preprocessing, X_train, y_train):
    learning_rate = trial.suggest_float("lgbm__learning_rate", 0.05, 0.2)
    num_leaves = trial.suggest_int("lgbm__num_leaves", 20, 80)
    n_estimators = trial.suggest_int("lgbm__n_estimators", 100, 300, step=50)
    pca_components = trial.suggest_float("pca__n_components", 0.90, 0.99)
    preprocessing_clone = clone(preprocessing)
    pipeline = make_pipeline(
        preprocessing_clone,
        PCA(n_components=pca_components),
        LGBMRegressor(
            random_state=42,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            n_jobs=-1,
            verbose=-1,
        )
    )
    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    return -scores.mean()


# =============================================================================
# STEP 7: Run Optuna Studies for PCA Models
# =============================================================================

pca_model_names = ["ridge_with_pca", "histgradientboosting_with_pca", "xgboost_with_pca", "lightgbm_with_pca"]
pca_objective_functions = {
    "ridge_with_pca": objective_ridge_pca,
    "histgradientboosting_with_pca": objective_hgb_pca,
    "xgboost_with_pca": objective_xgb_pca,
    "lightgbm_with_pca": objective_lgbm_pca,
}

pca_results = {}

for name in pca_model_names:
    base_name = name.replace("_with_pca", "")
    print(f"\n{'='*80}")
    print(f"Optimizing {name.upper()} - 10 trials")
    print(f"{'='*80}")

    study = optuna.create_study(
        direction="minimize",
        sampler=TPESampler(seed=42),
        study_name=f"{name}_study"
    )

    study.optimize(
        lambda trial: pca_objective_functions[name](trial, preprocessing, X_train, y_train),
        n_trials=10,
        show_progress_bar=True
    )

    cv_mae_pca = study.best_value
    print(f"\nBest {name.upper()} CV MAE: ${cv_mae_pca:,.2f}")
    print(f"Best params: {study.best_params}")

    best_params = study.best_params
    preprocessing_clone = clone(preprocessing)

    if base_name == "ridge":
        final_model = make_pipeline(
            preprocessing_clone,
            PCA(n_components=best_params["pca__n_components"]),
            Ridge(alpha=best_params["ridge__alpha"])
        )
    elif base_name == "histgradientboosting":
        final_model = make_pipeline(
            preprocessing_clone,
            PCA(n_components=best_params["pca__n_components"]),
            HistGradientBoostingRegressor(
                learning_rate=best_params["hgb__learning_rate"],
                max_depth=best_params["hgb__max_depth"],
                random_state=42
            )
        )
    elif base_name == "xgboost":
        final_model = make_pipeline(
            preprocessing_clone,
            PCA(n_components=best_params["pca__n_components"]),
            XGBRegressor(
                objective="reg:squarederror",
                random_state=42,
                n_estimators=best_params["xgb__n_estimators"],
                learning_rate=best_params["xgb__learning_rate"],
                max_depth=best_params["xgb__max_depth"],
                tree_method="hist",
                n_jobs=-1,
            )
        )
    elif base_name == "lightgbm":
        final_model = make_pipeline(
            preprocessing_clone,
            PCA(n_components=best_params["pca__n_components"]),
            LGBMRegressor(
                random_state=42,
                n_estimators=best_params["lgbm__n_estimators"],
                learning_rate=best_params["lgbm__learning_rate"],
                num_leaves=best_params["lgbm__num_leaves"],
                n_jobs=-1,
                verbose=-1,
            )
        )

    final_model.fit(X_train, y_train)

    y_pred = final_model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    print(f"{name} Test MAE: ${test_mae:,.2f}")

    pca_results[name] = {"pipeline": final_model, "test_mae": test_mae, "cv_mae": cv_mae_pca}

    with mlflow.start_run(run_name=f"{name}_optuna"):
        mlflow.log_param("model_family", base_name)
        mlflow.log_param("uses_pca", True)
        mlflow.log_params(best_params)
        mlflow.log_metric("cv_MAE", cv_mae_pca)
        mlflow.log_metric("test_MAE", test_mae)

        signature = infer_signature(X_train, final_model.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=final_model,
            artifact_path="housing_model_with_pca",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{base_name}_pipeline_with_pca_optuna",
        )

print("\n✓ STEP 7: All 4 PCA models optimized and logged.")


# =============================================================================
# STEP 8: Choose GLOBAL Best Model
# =============================================================================

all_results = {}
all_results.update(results)
all_results.update(pca_results)

global_best_name = min(all_results, key=lambda k: all_results[k]["test_mae"])
global_best_mae = all_results[global_best_name]["test_mae"]
global_best_cv_mae = all_results[global_best_name]["cv_mae"]
global_best_pipeline = all_results[global_best_name]["pipeline"]

uses_pca = "with_pca" in global_best_name

print("\n" + "=" * 80)
print("GLOBAL BEST MODEL (ACROSS 8 CANDIDATES)")
print("=" * 80)
print(f"Global best model key: {global_best_name}")
print(f"Global best CV MAE:    ${global_best_cv_mae:,.2f}")
print(f"Global best Test MAE:  ${global_best_mae:,.2f}")
print(f"Uses PCA:               {uses_pca}")


# ===========================================================================

Traceback (most recent call last):
  File "C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\store\tracking\file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\store\tracking\file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\store\tracking\file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

✓ STEP 1: Preprocessing pipeline created.
✓ STEP 2: Stratified split done. Train size: 16512, Test size: 4128
✓ STEP 3: MLflow configured.

Optimizing RIDGE (NO PCA) - 10 trials


  0%|          | 0/10 [00:00<?, ?it/s]


Best RIDGE CV MAE: $51,138.10
Best params: {'ridge__alpha': 0.14936568554617632}
ridge (no PCA) Test MAE: $52,372.31


C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


...

lightgbm_with_pca Test MAE: $34,714.91


C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


C:\Users\ASSUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Successfully registered model 'lightgbm_pipeline_with_pca_optuna'.
Created version '1' of model 'lightgbm_pipeline_with_pca_optuna'.



✓ STEP 7: All 4 PCA models optimized and logged.

GLOBAL BEST MODEL (ACROSS 8 CANDIDATES)
Global best model key: xgboost
Global best CV MAE:    $28,986.41
Global best Test MAE:  $28,384.85
Uses PCA:               False

--------------------------------------------------------------------------------
Saving and reloading GLOBAL best model...
--------------------------------------------------------------------------------


NameError: name 'base_folder' is not defined

In [9]:
# STEP 9: Save, Load, and Compare the GLOBAL Best Model
# =============================================================================

def save_model(model, filename="global_best_model_optuna.pkl"):
    joblib.dump(model, filename)
    print(f"✓ Model saved to {filename}")

print("\n" + "-" * 80)
print("Saving and reloading GLOBAL best model...")
print("-" * 80)

save_model(
    global_best_pipeline,
    filename=MODELS_DIR / "global_best_model_optuna.pkl"
)


print("\nDone:")
print(f"- GLOBAL best model key: {global_best_name}")
print(f"- GLOBAL best CV MAE:    ${global_best_cv_mae:,.2f}")
print(f"- GLOBAL best Test MAE:  ${global_best_mae:,.2f}")

end_time = time.monotonic()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = elapsed_time % 60
print(f"Elapsed time: {minutes} minutes and {seconds:.2f} seconds")


--------------------------------------------------------------------------------
Saving and reloading GLOBAL best model...
--------------------------------------------------------------------------------
✓ Model saved to ..\models\global_best_model_optuna.pkl

Done:
- GLOBAL best model key: xgboost
- GLOBAL best CV MAE:    $28,986.41
- GLOBAL best Test MAE:  $28,384.85
Elapsed time: 6 minutes and 53.89 seconds
